In [32]:
import sklearn as sks
import tensorflow as tf
import numpy as np
import os
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
import numpy as np
from keras.preprocessing import image
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Input
from keras.layers import Dense, Conv2D, Flatten, ZeroPadding2D 
from keras.layers import MaxPooling2D, Dropout, Input, BatchNormalization, Activation
from keras import regularizers
import splitfolders

os.chdir(r'C:\Users\python\Desktop\Cancer_Detection\Lung_Cancer_1')

In [33]:
img_height, img_width = 254,254
nb_train_samples = 65
nb_validation_samples = 10
epochs = 5
batch_size = 5  
pic_index = 0


In [34]:
os.chdir(r'C:\Users\python\Desktop\Cancer_Detection\Lung_Cancer_1\LungCancerDataset_stratified')

In [35]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',classes = ['normal', 'malignant', 'benign'],
    target_size=(img_height, img_width),batch_size=batch_size, class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    'val',classes = ['normal', 'malignant', 'benign'],
    target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')



Found 767 images belonging to 3 classes.
Found 330 images belonging to 3 classes.


In [36]:
input_shape=(224,224,3)
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=input_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add flatten layer
model.add(Flatten())

# Add dense/dropout layers + activation functions

model.add(Dense(128))
model.add(Activation('relu'))# relu activation function for deep layer
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))# sigmoid activation function for last layer for binary classification


In [37]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [42]:
from tensorflow.python.keras.layers import Lambda;

def norm(fc2):

    fc2_norm = K.l2_normalize(fc2, axis = 3);
    illum_est = tf.reduce_sum(fc2_norm, axis = (1, 2));
    illum_est = K.l2_normalize(illum_est);

    return illum_est;

illum_est = Lambda(norm)(fc2);

NameError: name 'fc2' is not defined

In [41]:
# x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: [[[[0.49821788]
   [0.49821788]
   [0.49821788]
   ...
   [0.49821788]
   [0.49821788]
   [0.49821788]]

  [[0.49821788]
   [0.49821788]
   [0.49821788]
   ...
   [0.49821788]
   [0.49821788]
   [0.49821788]]

  [[0.49821788]
   [0.49821788]
   [0.49821788]
   ...
   [0.49821788]
   [0.49821788]
   [0.49821788]]

  ...

  [[0.49821788]
   [0.49821788]
   [0.49821788]
   ...
   [0.49821788]
   [0.49821788]
   [0.49821788]]

  [[0.49821788]
   [0.49821788]
   [0.49821788]
   ...
   [0.49821788]
   [0.49821788]
   [0.49821788]]

  [[0.49821788]
   [0.49821788]
   [0.49821788]
   ...
   [0.49821788]
   [0.49821788]
   [0.49821788]]]]

In [38]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


In [ ]:
os.chdir(r"C:\Users\python\Desktop\Cancer_Detection\Lung_Cancer_1\LungCancerDataset_stratified\val\malignant")
lol = os.listdir()

In [ ]:
import numpy as np
random = np.random.randint(0,len(lol)) 
img_path = lol[random]  
# img_path2 = '/content/drive/MyDrive/Skin_Cancer/Benign/1792.jpg'
img = image.load_img(img_path, target_size=(img_width, img_height))
# img2 = image.load_img(img_path2, target_size=(img_width, img_height))


In [ ]:
# Malignant
img = image.img_to_array(img)
x = np.expand_dims(img, axis=0) * 1./255
score = model.predict(x)
print('Predicted of Malignant:', score[0])
print(score)

In [ ]:
# Benign
img = image.img_to_array(img2)
x = np.expand_dims(img, axis=0) * 1./255
score = model.predict(x)
print('Predicted:', score[0], 'Benign' if score > .50 and score < .90 else 'Malignant')